In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
pip install sastrawi

In [ ]:
pip install swifter

In [ ]:

import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
import itertools
import pandas as pd
import pickle
from IPython.display import display, clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rawdf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Kebakaran plumpang.csv')

#rawdf_10 = rawdf[rawdf["id_data"] < 124135]
#rawdf_10.head()

tweet_text = rawdf['Tweet']

print(rawdf.head(5))



                                               Tweet
0   musniumar Tenang aja proses legislasi DPR gak...
1   musniumar Kalo pemprov DKI dan  aniesbaswedan...
2  Bos Pertamina Ungkap Awal Mula Warga Padati La...
3  Depo Plumpang Tidak Bisa Segera Dipindah Dirut...
4  Depo Plumpang Tidak Bisa Segera Dipindah Dirut...


In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
tweet_text =tweet_text.str.lower()


print('Case Folding Result : \n')
print(tweet_text.head(5))
print('\n\n\n')

Case Folding Result : 

0     musniumar tenang aja proses legislasi dpr gak...
1     musniumar kalo pemprov dki dan  aniesbaswedan...
2    bos pertamina ungkap awal mula warga padati la...
3    depo plumpang tidak bisa segera dipindah dirut...
4    depo plumpang tidak bisa segera dipindah dirut...
Name: Tweet, dtype: object






In [ ]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
tweet_text = tweet_text.apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

tweet_text = tweet_text.apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

tweet_text = tweet_text.apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

tweet_text = tweet_text.apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

tweet_text = tweet_text.apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

tweet_text = tweet_text.apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

rawdf['token']= tweet_text.apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(rawdf['token'].head(5))
print('\n\n\n')


Tokenizing Result : 

0    [musniumar, tenang, aja, proses, legislasi, dp...
1    [musniumar, kalo, pemprov, dki, dan, aniesbasw...
2    [bos, pertamina, ungkap, awal, mula, warga, pa...
3    [depo, plumpang, tidak, bisa, segera, dipindah...
4    [depo, plumpang, tidak, bisa, segera, dipindah...
Name: token, dtype: object






In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
#txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
#list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

rawdf['tweet_tokens_WSW'] = rawdf['token'].apply(stopwords_removal) 


print(rawdf['tweet_tokens_WSW'].head(5))

0    [musniumar, tenang, proses, legislasi, dpr, se...
1    [musniumar, pemprov, dki, aniesbaswedanpeduli,...
2    [bos, pertamina, warga, padati, lahan, depo, p...
3    [depo, plumpang, dipindah, dirut, pertamina, b...
4    [depo, plumpang, dipindah, dirut, pertamina, b...
Name: tweet_tokens_WSW, dtype: object


In [ ]:
### --- Stemming --- ####

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in rawdf['tweet_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

rawdf['tweet_tokens_stemmed'] = rawdf['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)
print(rawdf['tweet_tokens_stemmed'])

Streaming output truncated to the last 5000 lines.
hilangnya : hilang
tingkatan : tingkat
pertamiina : pertamiina
power : power
gank : gank
fatal : fatal
ulang : ulang
perthankan : perthankan
instalansi : instalansi
dah : dah
trilyun : trilyun
akumulasi : akumulasi
balikpapan : balikpapan
mengeklik : klik
bio : bio
berati : berat
lazismu : lazismu
memberiuntuknegeri : memberiuntuknegeri
bcoplfybg : bcoplfybg
wietjk : wietjk
joshgrobax : joshgrobax
cilacap : cilacap
tinggalku : tinggal
deket : deket
evakuasi : evakuasi
karna : karna
tebal : tebal
itupun : itu
hitungan : hitung
wajar : wajar
antisipasi : antisipasi
fikir : fikir
isi : isi
kantong : kantong
migran : migran
ingatkan : ingat
benefit : benefit
korupsi : korupsi
terusir : usir
rugi : rugi
pencuri : curi
legalisir : legalisir
ngapunten : ngapunten
dilempari : lempari
scr : scr
moral : moral
ngeri : ngeri
terbitkan : terbit
curi : curi
harta : harta
gadainyawademiberkuasa : gadainyawademiberkuasa
mata : mata
erugian : erugian
d

Pandas Apply:   0%|          | 0/2205 [00:00<?, ?it/s]

0       [musniumar, tenang, proses, legislasi, dpr, tu...
1       [musniumar, pemprov, dki, aniesbaswedanpeduli,...
2       [bos, pertamina, warga, padat, lahan, depo, pl...
3       [depo, plumpang, pindah, dirut, pertamina, buf...
4       [depo, plumpang, pindah, dirut, pertamina, buf...
                              ...                        
2200    [jasinskm, azgeopolitics, the, consequence, of...
2201    [midwest, thinkin, kjsmom, she, is, repeat, of...
2202    [leoneissues, have, been, there, before, it, n...
2203    [the, city, of, calgary, is, asking, council, ...
2204    [blackintheempir, it, will, go, on, past, this...
Name: tweet_tokens_stemmed, Length: 2205, dtype: object


In [ ]:
rawdf.to_csv("/content/drive/MyDrive/Colab Notebooks/dataset/preprocessing kebakaran plumpang.csv")

In [ ]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

for i in tweet_text:
  # stemming process
  sentence = i
  output   = stemmer.stem(sentence)

  print(output)
  # ekonomi indonesia sedang dalam tumbuh yang bangga


musniumar tenang aja proses legislasi dpr gak tuju pindah depo plumpang minus pd nasdem dan pks perlahan usut yang asih imb hehehe
musniumar kalo pemprov dki dan aniesbaswedanpeduli dg selamat warga dki dan gga fasilitas imb buffer zone depo plumpang ga akan jadi padat duduk klo bufferzone tetap jaga klo pun jadi celaka yg sama seperti kemarin korban ga akan seperti itu loe dongok
bos pertamina ungkap awal mula warga padat lahan depo plumpang nyata begini https co gdutluyw
depo plumpang tidak bisa segera pindah dirut pertamina buffer zone perlu cepat bangun https co releixaxot
depo plumpang tidak bisa segera pindah dirut pertamina buffer zone perlu cepat bangun https co sktvzcspu
depo plumpang tidak bisa segera pindah dirut pertamina buffer zone perlu cepat bangun https co wkaikjdqe
depo plumpang tidak bisa segera pindah dirut pertamina buffer zone perlu cepat bangun https co bxgyqyhfmm
depo plumpang tidak bisa segera pindah dirut pertamina buffer zone perlu cepat bangun https co udelr

In [ ]:
import pandas as pd 
import numpy as np

TWEET_DATA = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/preprocessing kebakaran plumpang.csv", usecols=["tweet_tokens_stemmed"])
TWEET_DATA.columns = ["tweet"]

TWEET_DATA.head()

,tweet
0,"['musniumar', 'tenang', 'proses', 'legislasi',..."
1,"['musniumar', 'pemprov', 'dki', 'aniesbaswedan..."
2,"['bos', 'pertamina', 'warga', 'padat', 'lahan'..."
3,"['depo', 'plumpang', 'pindah', 'dirut', 'perta..."
4,"['depo', 'plumpang', 'pindah', 'dirut', 'perta..."


In [ ]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet"].apply(convert_text_list)


print(TWEET_DATA["tweet_list"][90])

print("\ntype : ", type(TWEET_DATA["tweet_list"][90]))

['depo', 'pertamina', 'plumpang', 'jakarta', 'utara', 'diri', 'an', 'milik', 'lahan', 'cuat', 'bakar', 'hebat', 'landa', 'depo', 'pertamina', 'plumpang', 'jumat', 'malam', 'https', 'co', 'fbnugla', 'infografis', 'cnnindonesia', 'https', 'co', 'ytyelnimuw']

type :  <class 'list'>


In [ ]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)

TWEET_DATA["TF_dict"].head()

0    {'musniumar': 0.05555555555555555, 'tenang': 0...
1    {'musniumar': 0.043478260869565216, 'pemprov':...
2    {'bos': 0.1, 'pertamina': 0.1, 'warga': 0.1, '...
3    {'depo': 0.08333333333333333, 'plumpang': 0.08...
4    {'depo': 0.08333333333333333, 'plumpang': 0.08...
Name: TF_dict, dtype: object

In [ ]:
index = 90

print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

                term 	 TF

                depo 	 0.07692307692307693
           pertamina 	 0.07692307692307693
            plumpang 	 0.07692307692307693
             jakarta 	 0.038461538461538464
               utara 	 0.038461538461538464
                diri 	 0.038461538461538464
                  an 	 0.038461538461538464
               milik 	 0.038461538461538464
               lahan 	 0.038461538461538464
                cuat 	 0.038461538461538464
               bakar 	 0.038461538461538464
               hebat 	 0.038461538461538464
               landa 	 0.038461538461538464
               jumat 	 0.038461538461538464
               malam 	 0.038461538461538464
               https 	 0.07692307692307693
                  co 	 0.07692307692307693
             fbnugla 	 0.038461538461538464
          infografis 	 0.038461538461538464
        cnnindonesia 	 0.038461538461538464
          ytyelnimuw 	 0.038461538461538464


In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])

In [ ]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)

In [ ]:
# Check TF-IDF result
index = 90

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])


                term 	         TF 	              TF-IDF

                depo 	 0.07692307692307693 	 0.039410652566047776
           pertamina 	 0.07692307692307693 	 0.049607466206713066
            plumpang 	 0.07692307692307693 	 0.02642160513659154
             jakarta 	 0.038461538461538464 	 0.09212222614699501
               utara 	 0.038461538461538464 	 0.10548675885691099
                diri 	 0.038461538461538464 	 0.1754995604596845
                  an 	 0.038461538461538464 	 0.1974435934776696
               milik 	 0.038461538461538464 	 0.15721403366295084
               lahan 	 0.038461538461538464 	 0.14968539608117795
                cuat 	 0.038461538461538464 	 0.22718166610203427
               bakar 	 0.038461538461538464 	 0.026260398745912197
               hebat 	 0.038461538461538464 	 0.18945746406312175
               landa 	 0.038461538461538464 	 0.2427764779523483
               jumat 	 0.038461538461538464 	 0.16793377991175934
               malam 	

In [ ]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.01908227037642722, 0.0, 0.0, 0.02846324907547895, 0.0, 0.0, 0.0, 0.0, 0.10740509361645009, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15600742553662178, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [ ]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)



,term,rank
4,pertamina,56.829340
9,minyak,54.891254
13,kilang,54.807580
8,pindah,54.461183
5,bakar,54.208457
6,korban,50.209250
3,depo,48.724094
2,https,46.647126
14,sebab,45.842730
1,co,45.611089


In [ ]:
ranking.to_csv("/content/drive/MyDrive/Colab Notebooks/dataset/ranking kebakaran plumpang.csv")